# TREC 2006

In [1]:
import numpy as np
import pandas as pd
import email
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

data_dir="./trec06p/"
full_index_dir="./trec06p/full/index"

full_index=pd.read_csv(full_index_dir,header=None,names=["label","index"],sep=" ")
full_index["index"]=full_index["index"].apply(lambda x:data_dir+x[3:])
full_index.tail()

,label,index
37817,spam,./trec06p/data/126/017
37818,spam,./trec06p/data/126/018
37819,spam,./trec06p/data/126/019
37820,spam,./trec06p/data/126/020
37821,spam,./trec06p/data/126/021


In [2]:
Errorlst=[]
EmailInfo=pd.DataFrame({},columns=["email_id","parts","attachments","html","subject","body","links","spam"])

for i,row in full_index.iterrows():

    try:
        with open(row["index"],"r") as f:
            raw_email=f.read()
    except:
        Errorlst.append((i,row,"UnicodeDecodeError"))
        continue

    parsed_email = email.message_from_string(raw_email)
    attachments=0
    parts=0
    html=0

    try:
        if parsed_email.is_multipart():
            for part in parsed_email.walk():
                ctype = part.get_content_type()
                cdispo = str(part.get('Content-Disposition'))
                
                parts+=1

                if 'attachment' in cdispo:
                    attachments+=1
                    
                if ctype == 'text/html':
                    html=1

                # skip any text/plain (txt) attachments
                if (ctype == 'text/plain') and 'attachment' not in cdispo:
                    body = part.get_payload(decode=True)  # decode

        # not multipart - i.e. plain text, no attachments, keeping fingers crossed
        else:
            body = parsed_email.get_payload(decode=True)
            ctype = parsed_email.get_content_type()
            cdispo = str(parsed_email.get('Content-Disposition'))
            
            if 'attachment' in cdispo:
                attachments+=1
            
            if ctype == 'text/html':
                html=1
        
        subject=parsed_email["Subject"]
        
    except:
        Errorlst.append((i,row,"binascii.Error"))
        continue

    soup = BeautifulSoup(body)
    
    email_id = row["index"][-7:]
    label = 1 if row["label"]=="spam" else 0
    links=len(soup.find_all("a"))
    
    Info=pd.DataFrame([[email_id,parts,attachments,html,subject,body,links,label]],columns=EmailInfo.columns)
    EmailInfo=pd.concat([EmailInfo,Info],axis=0,ignore_index=True)
    
    if i%10000==0:
        print("{} done.".format(i))
        
        
EmailInfo.tail(10)
    

0 done.
10000 done.
20000 done.
30000 done.


,email_id,parts,attachments,html,subject,body,links,spam
35278,126/011,0,0,0,This will surely help.,b'F A S T T R A C K D E G R E E P R O G...,0,1
35279,126/012,2,0,0,Please update your online banking !,b'\n',0,1
35280,126/013,3,0,1,Ephedra for you again,b'%TXT_ADD\n',0,1
35281,126/015,0,0,0,Get your wanted degree now,"b""Genuine College Degree in 2 Weeks! \n\nHave ...",0,1
35282,126/016,0,0,0,{THE_SUBJECT},b'bla bla bla\neee\ne\n\n\n\n\n\n\n\nrererreer...,0,1
35283,126/017,0,0,0,RE: Watcher daily news,b'Great News Expec ted!\n\nInfinex Ventures In...,0,1
35284,126/018,0,0,0,RE: Number 1 market Picks,b'The OIL sector is going crazy. This is our w...,0,1
35285,126/019,0,0,0,All products for your health!,"b""http://vdtobj.docscan.info/?23759301\n\nSuff...",0,1
35286,126/020,0,0,0,"Its here, now.","b""U N I V E R S I T Y D I P L O M A S\n\nDo y...",0,1
35287,126/021,3,0,1,This diet plan is over the roof,"b'but moat , coverall be cytochemistry be plan...",0,1


In [3]:
EmailInfo[["parts","attachments","html","links","spam"]]=\
EmailInfo[["parts","attachments","html","links","spam"]].astype("int32")
i,EmailInfo.dtypes, EmailInfo.describe()

(37821,
 email_id       object
 parts           int32
 attachments     int32
 html            int32
 subject        object
 body           object
 links           int32
 spam            int32
 dtype: object,
               parts   attachments          html         links          spam
 count  35288.000000  35288.000000  35288.000000  35288.000000  35288.000000
 mean       0.899853      0.007311      0.328242      0.288257      0.643533
 std        1.405208      0.100459      0.469580      2.523199      0.478962
 min        0.000000      0.000000      0.000000      0.000000      0.000000
 25%        0.000000      0.000000      0.000000      0.000000      0.000000
 50%        0.000000      0.000000      0.000000      0.000000      1.000000
 75%        3.000000      0.000000      1.000000      0.000000      1.000000
 max       35.000000      4.000000      1.000000    227.000000      1.000000)

In [6]:
# EmailInfo.to_csv("email_06.csv")
len(Errorlst)

2534

# Structural Info

In [7]:
EmailInfo=pd.read_csv("email_06.csv")

StructuralInfo=pd.DataFrame({"multipart":EmailInfo["parts"],"html":EmailInfo["html"],\
                             "links":EmailInfo["links"],"attachments":EmailInfo["attachments"],\
                             "spam":EmailInfo["spam"]})
StructuralInfo["multipart"]=(StructuralInfo["multipart"]>0).astype(int)
StructuralInfo["links"]=(StructuralInfo["links"]>0).astype(int)
StructuralInfo["attachments"]=(StructuralInfo["attachments"]>0).astype(int)

StructuralInfo.describe()

,multipart,html,links,attachments,spam
count,35288.000000,35288.000000,35288.000000,35288.000000,35288.000000
mean,0.307413,0.328242,0.067643,0.006319,0.643533
std,0.461429,0.469580,0.251137,0.079244,0.478962
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# StructuralInfo.to_csv("email_06_structure.csv")

# Subject Feature

In [10]:
import numpy as np
import pandas as pd
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import sys 
sys.path.append("../../")
from E4525_ML import text

import warnings
warnings.filterwarnings("ignore")

EmailInfo=pd.read_csv("email_06.csv")
Subject_content=EmailInfo[["subject","spam"]]
Subject_content.fillna(value="",inplace=True)

In [16]:
data_dir = "subject_feature"

set_vectorizer_filename=   data_dir+"/set_vectorizer.p"
count_vectorizer_filename=   data_dir+"/count_vectorizer.p"
tfidf_vectorizer_filename=   data_dir+"/tfidf_vectorizer.p"

count_Vectorizer=pickle.load(open( count_vectorizer_filename, "rb" ) )
set_Vectorizer=pickle.load(open( set_vectorizer_filename, "rb" ) )
tfidf_Vectorizer=pickle.load(open( tfidf_vectorizer_filename, "rb" ) )

In [17]:
%time X_set=set_Vectorizer.transform(Subject_content["subject"])
%time X_count=count_Vectorizer.transform(Subject_content["subject"])
%time X_tfidf=tfidf_Vectorizer.transform(Subject_content["subject"])
print(X_set.shape,X_count.shape,X_tfidf.shape,)

Wall time: 6.57 s
Wall time: 6.65 s
Wall time: 6.73 s
(35288, 36750) (35288, 36750) (35288, 36750)


In [18]:
set_features_filename=     data_dir+"/set_features06.p"
count_features_filename=     data_dir+"/count_features06.p"
tfidf_features_filename=     data_dir+"/tfidf_features06.p"
labels_filename=     data_dir+"/labels06.p"

pickle.dump(X_set,         open( set_features_filename, "wb" ) )
pickle.dump(X_count,             open( count_features_filename, "wb" ) )
pickle.dump(X_tfidf,             open( tfidf_features_filename, "wb" ) )
pickle.dump(Subject_content["spam"].values,         open( labels_filename, "wb" ) )

# Message Feature

In [19]:
data_dir = "message_feature"

set_vectorizer_filename=   data_dir+"/set_vectorizer_-1.p"
count_vectorizer_filename=   data_dir+"/count_vectorizer_-1.p"
tfidf_vectorizer_filename=   data_dir+"/tfidf_vectorizer_-1.p"

count_Vectorizer=pickle.load(open( count_vectorizer_filename, "rb" ) )
set_Vectorizer=pickle.load(open( set_vectorizer_filename, "rb" ) )
tfidf_Vectorizer=pickle.load(open( tfidf_vectorizer_filename, "rb" ) )

In [ ]:
%time X_set=set_Vectorizer.transform(EmailInfo["body"])
%time X_count=count_Vectorizer.transform(EmailInfo["body"])
%time X_tfidf=tfidf_Vectorizer.transform(EmailInfo["body"])
print(X_set.shape,X_count.shape,X_tfidf.shape,)

Wall time: 2h 59min 1s


In [ ]:
set_features_filename=     data_dir+"/set_features06.p"
count_features_filename=     data_dir+"/count_features06.p"
tfidf_features_filename=     data_dir+"/tfidf_features06.p"
labels_filename=     data_dir+"/labels06.p"

pickle.dump(X_set,         open( set_features_filename, "wb" ) )
pickle.dump(X_count,             open( count_features_filename, "wb" ) )
pickle.dump(X_tfidf,             open( tfidf_features_filename, "wb" ) )
pickle.dump(EmailInfo["spam"].values,         open( labels_filename, "wb" ) )